In [36]:
import sys
import os

# Get the current working directory
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
print(parent_directory)

if 'TimeXer' in parent_directory:
    # Add the parent directory to sys.path
    sys.path.append(parent_directory)
    os.chdir(parent_directory)
%load_ext autoreload
%autoreload 2



/home/robers/projects
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
from utils.database import connectionSQL
import pandas as pd

engine = connectionSQL("PROMART_PRD", 'cfg/secrets.yaml')

# query = """
# SELECT DISTINCT [Type_query]
# FROM [PROMART_PRD].[dbo].[t_ML_meteoswiss_weather_forecast];"""

# m_suisse_types = pd.read_sql_query(query, con=engine)

# query = """
# SELECT DISTINCT [Type_query]
# FROM [PROMART_PRD].[dbo].[t_ML_weather_data];"""

# weather_data_types = pd.read_sql_query(query, con=engine)

In [38]:
# print(m_suisse_types['Type_query'])
# print(weather_data_types['Type_query'])
m_suisse_type = 'icon_ch2_0'
weather_data_type = 'matics-mos-flex'

In [ ]:
Abbreviations = ['ABO', 'PAY']
start_date = '2024-01-01'
end_date = '2024-01-31'

# Convert the list of abbreviations into a SQL-compatible string
abbreviation_list = "', '".join(Abbreviations)

query = f"""
SELECT
      [Abbreviation],
      [Delivery_date],
      [Issue_date],
      [Type_query],
      [rain {{mm/h}}],
      [temperature {{Celsius}}],
      [relative humidity{{%}}],
      [radiation globale {{W/m2}}]
  FROM [PROMART_PRD].[dbo].[t_ML_weather_data]
  WHERE [Type_query] = '{weather_data_type}' 
    AND [Abbreviation] IN ('{abbreviation_list}')
    AND [Delivery_date] BETWEEN '{start_date}' AND '{end_date}'
"""

# Execute the query and load the results into a DataFrame
df = pd.read_sql_query(query, con=engine)
df.drop('Type_query', axis=1, inplace=True)
print(len(df))


AttributeError: 'DataFrame' object has no attribute 'dropcolumns'

In [ ]:
# endogenous variables: load from gb 
# exogenous variables: temperature, rain, radiation, calendar_date, day_of_week, is_holiday

from utils.dataset import remove_duplicates_keep_latest
print(len(df['Delivery_date'].unique()), len(df['Issue_date'].unique()))
df_unique = remove_duplicates_keep_latest(df, group_cols=['Abbreviation', 'Delivery_date'], sort_col='Issue_date')
print(len(df_unique['Delivery_date'].unique()), len(df_unique['Issue_date'].unique()))

2881 73
2881 61


In [ ]:
query = f"""
SELECT
      [Delivery_date],
      [Issue_date],
      [CMO],
      [MAG],
      [Bilan],
      [Validation]
FROM [PROMART_PRD].[dbo].[t_ML_validation_gb_consumption] 
WHERE ([Validation] = '100' OR [Validation] = '110')
  AND [Delivery_date] BETWEEN '{start_date}' AND '{end_date}'"""

df_load = pd.read_sql_query(query, con=engine)

In [ ]:
df_load_unique = remove_duplicates_keep_latest(df_load, group_cols=['Delivery_date'], sort_col='Issue_date')


In [ ]:
df_unique_merged.columns


Index(['Abbreviation', 'Delivery_date', 'Issue_date', 'Type_query',
       'rain {mm/h}', 'temperature {Celsius}', 'relative humidity{%}',
       'radiation globale {W/m2}', 'Bilan'],
      dtype='object')

In [ ]:
from utils.dataset import append_abbreviation_to_columns
# Example columns to update
columns_to_update = ['rain {mm/h}', 'temperature {Celsius}', 'relative humidity{%}', 'radiation globale {W/m2}']

# Apply the function to the dataframe
df_unique_noabbr = append_abbreviation_to_columns(df_unique, columns_to_update)


In [ ]:
print(len(df_unique_noabbr))
print(len(df_unique_noabbr['Delivery_date'].unique()))
df_unique_noabbr.columns

2881
2881


Index(['Delivery_date', 'ABO rain {mm/h}', 'ABO temperature {Celsius}',
       'ABO relative humidity{%}', 'ABO radiation globale {W/m2}',
       'PAY rain {mm/h}', 'PAY temperature {Celsius}',
       'PAY relative humidity{%}', 'PAY radiation globale {W/m2}'],
      dtype='object')

In [ ]:
# Ensure both dataframes are sorted by 'Delivery_date'
df_unique_noabbr = df_unique_noabbr.sort_values('Delivery_date')
df_load_unique = df_load_unique.sort_values('Delivery_date')

# Perform an inner merge to keep only matching 'Delivery_date' rows in both dataframes
df_unique_merged = pd.merge(
    df_unique_noabbr, 
    df_load_unique[['Delivery_date', 'Bilan']], 
    on='Delivery_date', 
    how='inner'
)

print(len(df_unique_merged))